In [ ]:
#| default_exp weather

Weather: Lets give them something to talk about

In [ ]:
#| export
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

#custom
import inspect
import copy
import colorsys
from importlib import resources

from HexMagic.styles import StyleCSS, SVGBuilder, SVGLayer, SVGPatternLoader, preview, app, StyleDemo, LayerAnimation
from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexWrapper, HexPosition, hexBackground, HexRegion, windy_edge,unique_windy_edge
from HexMagic.terrain import  TerraDemo, Terrain, GeoBounds, ClimatePreset
from HexMagic.terrainpatterns import TerrainPatterns
#from HexMagic.river import River, SoilSystem, ErosionModel

In [ ]:
#| export
from HexMagic.terrainpatterns import TerrainPatterns

In [ ]:
#| export
#from HexMagic.river import River, RiverDemo

In [ ]:
#| export
import math
from collections import deque

In [ ]:
#| export
import orographic_precipitation as op



So we are going to reimplement a great deal of what we had using the model based upon this link `https://journals.ametsoc.org/view/journals/atsc/61/12/1520-0469_2004_061_1377_altoop_2.0.co_2.xml` the goal is to find compontents that we can compute climates/biomes 

class Climate(Enum):
    MARINE = 0        # Ocean/sea
    FRESHWATER = 1    # Lakes/rivers
    TUNDRA = 2        # Cold, low precipitation
    DESERT = 3        # Hot/cold, very low precipitation
    GRASSLAND = 4     # Moderate temp, moderate precipitation
    FOREST = 5        # Moderate temp, high precipitation
    JUNGLE = 6        # Hot, very high precipitation
I know that we can use things like elevation, temperature, preciptation to start doing this. I don't know if there are other factors that would be helpful. one thing that is good to understand is our hexgrid system which has rings of hexes around any hex which makes it easy to compute things like shorest path or sampling. I am going to include some sample code here which shows how we average fields across our data.

`
@patch
def downsample_field(self: Terrain, field_array, sample_radius=2, method='weighted_avg'):
    """Downsample any field array using hex ring sampling.
    
    Args:
        field_array: numpy array of values to downsample (same length as hexes)
        sample_radius: How many rings to sample
        method: 'weighted_avg', 'max', 'min', or 'mode' (for categorical)
    
    Returns:
        Downsampled array matching new grid size
    """
    old_grid = self.hexGrid
    factor = sample_radius + 1
    new_rows = old_grid.nRows // factor
    new_cols = old_grid.nCols // factor
    
    if new_rows < 2 or new_cols < 2:
        print("Warning: sample_radius too large")
        return field_array
    
    # Calculate new grid size
    new_size = new_rows * new_cols
    new_field = np.zeros(new_size)
    
    for new_idx in range(new_size):
        new_row = new_idx // new_cols
        new_col = new_idx % new_cols
        
        old_row = new_row * factor
        old_col = new_col * factor
        
        if old_row >= old_grid.nRows or old_col >= old_grid.nCols:
            continue
            
        center_idx = old_row * old_grid.nCols + old_col
        
        # Collect values from rings
        values = []
        weights = []
        
        for ring_num in range(sample_radius + 1):
            weight = 1.0 / (ring_num + 1)
            
            if ring_num == 0:
                ring_hexes = [center_idx]
            else:
                ring_positions = HexPosition(0, 0, 0).ring(ring_num)
                ring_hexes = [old_grid.hexposition_to_index(hp, center_idx) 
                             for hp in ring_positions]
                ring_hexes = [h for h in ring_hexes if h >= 0]
            
            for hex_idx in ring_hexes:
                values.append(field_array[hex_idx])
                weights.append(weight)
        
        if not values:
            continue
            
        values = np.array(values)
        weights = np.array(weights)
        
        # Apply aggregation method
        if method == 'weighted_avg':
            new_field[new_idx] = np.average(values, weights=weights)
        elif method == 'max':
            new_field[new_idx] = np.max(values)
        elif method == 'min':
            new_field[new_idx] = np.min(values)
        elif method == 'mode':
            # For categorical data - weighted mode
            unique_vals = np.unique(values)
            best_val = unique_vals[0]
            best_weight = 0
            for val in unique_vals:
                val_weight = np.sum(weights[values == val])
                if val_weight > best_weight:
                    best_weight = val_weight
                    best_val = val
            new_field[new_idx] = best_val
    
    return new_field
`
@patch
def downsample_ring(self: Terrain, sample_radius=2, method='weighted_avg'):
    """Downsample terrain using hex ring sampling.
    
    Args:
        sample_radius: How many rings to sample (higher = more smoothing)
        method: 'weighted_avg', 'max', 'min', or 'mode' (for categorical)
    
    Returns:
        New Terrain with fewer hexes covering same area
    """
    old_grid = self.hexGrid
    
    # Calculate new grid dimensions
    factor = sample_radius + 1
    new_rows = old_grid.nRows // factor
    new_cols = old_grid.nCols // factor
    
    if new_rows < 2 or new_cols < 2:
        print("Warning: sample_radius too large for this terrain")
        return self
    
    # Keep same bounds, just use larger hex radius
    new_radius = old_grid.radius * factor
    
    # Create new terrain with same bounds but larger hexes
    new_terrain = Terrain(old_grid.bounds, radius=new_radius)
    
    # Copy style properties
    new_terrain.colorLevels = self.colorLevels.copy() if self.colorLevels else None
    new_terrain.elevationDelta = self.elevationDelta
    new_terrain.seaLevel = self.seaLevel
    
    # Use the downsample_field method to sample elevations
    new_terrain.elevations = self.downsample_field(
        self.elevations, 
        sample_radius=sample_radius, 
        method=method
    )
    
    # Add color styles
    if new_terrain.colorLevels:
        for color in new_terrain.colorLevels:
            new_terrain.hexGrid.builder.add_style(color)
    
    return new_terrain
    `
    So I think we need things that are going to create fields of data that we can then use to compute things like temperature, precipitation. we can take advantage of some of the existing mapping code to see progress. for intance this shows climate

    @patch
def add_climate_overlay(self: Terrain, layer_name="climate"):
    """Visualize climate zones with appropriate colors."""
    
    # Define colors for each climate type
    climate_colors = {
        Climate.MARINE.value: "#1e88e5",      # Blue - ocean
        Climate.FRESHWATER.value: "#42a5f5",  # Light blue - lakes
        Climate.TUNDRA.value: "#e3f2fd",      # Pale blue - tundra
        Climate.DESERT.value: "#fdd835",      # Yellow - desert
        Climate.GRASSLAND.value: "#9ccc65",   # Light green - grassland
        Climate.FOREST.value: "#2e7d32",      # Dark green - forest
        Climate.JUNGLE.value: "#1b5e20",      # Very dark green - jungle
    }
    
    if 'climate' not in self.fields:
        self.compute_climate()
    
    # Create patterns for each climate type
    patternGen = TerrainPatterns(self)
    colors = [climate_colors[i] for i in range(len(Climate))]
    patterns = patternGen.ballDensity(len(colors), fills=colors, prefix="climate")
    
    # Generate overlay
    climate_indices = self.fields['climate'].astype(int)
    overlay = self.makeOverlay(climate_indices, patterns)
    
    # Add to builder
    self.builder.adjust(layer_name, overlay)
    `

    Let's think about how we would go through things.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>

```json
{
  "id": "toolu_016pwTWX4YWFHRMBxHHm6k94",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://journals.ametsoc.org/view/journals/atsc/61/12/1520-0469_2004_061_1377_altoop_2.0.co_2.xml"
    }
  },
  "result": "\"# JavaScript is disabled\\n\\nIn order to continue, we need to verify that you're not a robot. This requires JavaScript. Enable JavaScript and then reload the page. \\n\""
}
```

</details>

Let me search for information about this paper and the Linear Theory of Orographic Precipitation model:No worries - I'm familiar with the Smith & Barstad (2004) model! And I see you already have the `orographic_precipitation` package installed. Let me help you think through this systematically.

**The Linear Theory of Orographic Precipitation** is a physics-based model that computes precipitation from:

1. **Terrain elevation** (the DEM)
2. **Wind speed and direction** 
3. **Atmospheric parameters** (moisture, stability, scale heights)
4. **Grid resolution** (dx, dy)

This is actually more principled than your current wind-tree approach because it uses Fourier transforms to solve the physics equations.

**For computing Climate/Biomes, you need these primary fields:**

| Field | Source | Notes |
|-------|--------|-------|
| **Elevation** | Already have | Your terrain |
| **Temperature** | Compute from lat + elevation | Lapse rate ~6.5°C/1000m |
| **Precipitation** | Smith-Barstad model | Uses `orographic_precipitation` |
| **Distance to coast** | BFS from ocean hexes | For continentality |
| **Latitude** | From bounds | Temperature gradient |

**Additional factors that could help:**

- **Seasonality** - Wet/dry seasons (could run model for summer vs winter wind patterns)
- **Continentality** - Distance from ocean affects temp range and moisture
- **Aspect/slope** - South-facing vs north-facing affects local temp

**Let's start by getting the Smith-Barstad model working with your hex grid.**

The key challenge: `compute_orographic_precip` expects a **2D numpy array** (regular grid), but your terrain uses a hex grid. You'll need to:

1. Convert hex elevations → 2D array
2. Run the orographic model
3. Sample results back to hex positions

Here's a sketch of the approach:

```python
@patch
def elevation_to_grid(self: Terrain):
    """Convert hex elevations to 2D numpy array for orographic model."""
    grid = self.hexGrid
    # Create 2D array matching hex grid dimensions
    elev_2d = self.elevations.reshape(grid.nRows, grid.nCols)
    return elev_2d

@patch  
def compute_orographic_rainfall(self: Terrain, 
                                 wind_speed=10.0,      # m/s
                                 wind_dir=270.0,       # degrees (270=westerly)
                                 latitude=37.0,        # for Coriolis
                                 precip_base=0.5,      # background precip mm/h
                                 annual_hours=8760):   # convert to annual mm
    """Compute precipitation using Smith-Barstad linear model."""
    import orographic_precipitation as op
    
    # Get 2D elevation array
    elev_2d = self.elevation_to_grid()
    
    # Grid spacing in meters (approximate from hex radius)
    dx = dy = self.hexGrid.radius * 1.5  # rough conversion
    
    # Run the model
    precip_rate = op.compute_orographic_precip(
        elev_2d, dx, dy,
        latitude=latitude,
        wind_speed=wind_speed,
        wind_dir=wind_dir,
        precip_base=precip_base,
        precip_min=0.01
    )
    
    # Convert mm/h to annual mm
    annual_precip = precip_rate * annual_hours
    
    # Flatten back to hex array
    self.fields['precipitation'] = annual_precip.flatten()
    
    return self.fields['precipitation']
```

**Questions before we dive in:**

1. What's your hex grid's approximate real-world resolution? (meters per hex)
2. Do you want to start with the basic model, or also include multiple wind directions right away?
3. Should we also refactor `compute_temperature` to be cleaner, or focus on precipitation first?

In [ ]:
#| export
@patch
def _compute_hex_coordinates(self: Terrain):
    """Compute lat/lon for each hex center."""
    if self.geo is None:
        return
    
    grid = self.hexGrid
    
    # Store lat/lon for each hex
    self.fields['latitude'] = np.zeros(len(self.elevations))
    self.fields['longitude'] = np.zeros(len(self.elevations))
    
    for idx in range(len(self.elevations)):
        row = idx // grid.nCols
        col = idx % grid.nCols
        
        # Normalize to [0, 1]
        row_norm = row / (grid.nRows - 1) if grid.nRows > 1 else 0.5
        col_norm = col / (grid.nCols - 1) if grid.nCols > 1 else 0.5
        
        # Map to geographic coordinates
        # Note: row 0 = top = max latitude
        self.fields['latitude'][idx] = self.geo.lat_max - row_norm * (self.geo.lat_max - self.geo.lat_min)
        self.fields['longitude'][idx] = self.geo.lon_min + col_norm * (self.geo.lon_max - self.geo.lon_min)

In [ ]:
#| export

@patch
def grid_resolution_meters(self: Terrain):
    """Calculate dx, dy in meters for the grid."""
    if self.geo is None:
        return None, None
    
    # Meters per degree at center latitude
    lat_center = (self.geo.lat_min + self.geo.lat_max) / 2
    lat_rad = math.radians(lat_center)
    
    meters_per_lat = 111132.92  # ~constant
    meters_per_lon = 111132.92 * math.cos(lat_rad)
    
    # Resolution per grid cell
    lat_span = self.geo.lat_max - self.geo.lat_min
    lon_span = self.geo.lon_max - self.geo.lon_min
    
    dy = (lat_span / self.hexGrid.nRows) * meters_per_lat
    dx = (lon_span / self.hexGrid.nCols) * meters_per_lon
    
    return dx, dy



In [ ]:
#| export
@patch
def compute_precipitation_sb(self: Terrain,
                              wind_speed=15.0,
                              wind_dir=270.0,
                              precip_base=0.3,
                              # Atmospheric parameters with typical values
                              nm=0.005,         # moist stability frequency [1/s]
                              hw=2500.0,        # water vapor scale height [m]
                              cw=0.005,         # uplift sensitivity [kg/m³]
                              conv_time=1000.0, # conversion time [s]
                              fall_time=1000.0 , # fallout time [s]
                              debug = False
                            
                             ):
    """
    Compute orographic precipitation using Smith-Barstad (2004) model.
    
    Args:
        wind_speed: Wind speed in m/s (typical: 10-20)
        wind_dir: Wind direction in degrees (270 = from west)
        precip_base: Background precipitation rate in mm/h (typical: 0.1-1.0)
        nm: Moist stability frequency [1/s] (typical: 0.005)
        hw: Water vapor scale height [m] (typical: 2000-3000)
        cw: Uplift sensitivity [kg/m³] (typical: 0.003-0.006)
        conv_time: Cloud water conversion time [s] (typical: 500-1500)
        fall_time: Hydrometeor fallout time [s] (typical: 500-1500)
    
    Returns:
        Annual precipitation in mm
    """
    
    
    if self.geo is None:
        raise ValueError("Geographic bounds not set. Use terrain.geo = GeoBounds(...)")
    
    # Get 2D elevation array
    grid = self.hexGrid
    elev_2d = self.elevations.reshape(grid.nRows, grid.nCols)
    
    # Get grid resolution in meters
    dx, dy = self.grid_resolution_meters()
    
    if debug:
        print(f"Grid: {grid.nRows} x {grid.nCols}")
        print(f"Resolution: dx={dx:.0f}m (~{dx/1000:.1f}km), dy={dy:.0f}m (~{dy/1000:.1f}km)")
        print(f"Latitude: {self.geo.lat_min}° to {self.geo.lat_max}° (center: {(self.geo.lat_min + self.geo.lat_max)/2:.1f}°)")
        print(f"Wind: {wind_speed} m/s from {wind_dir}°")
        print(f"Background precip: {precip_base} mm/h")
    
    # Run Smith-Barstad model with all required parameters
    precip_rate = op.compute_orographic_precip(
        elev_2d, dx, dy,
        latitude=(self.geo.lat_min + self.geo.lat_max) / 2,
        wind_speed=wind_speed,
        wind_dir=wind_dir,
        precip_base=precip_base,
        precip_min=0.01,
        nm=nm,
        hw=hw,
        cw=cw,
        conv_time=conv_time,
        fall_time=fall_time
    )
    
    # Convert mm/h to annual mm
    hours_per_year = 8760
    annual_precip = precip_rate * hours_per_year
    
    # Flatten and store
    self.fields['precipitation'] = annual_precip.flatten()
    self.fields['precip_rate_mmh'] = precip_rate.flatten()
    
    if debug:
        print(f"\nPrecipitation range: {annual_precip.min():.0f} - {annual_precip.max():.0f} mm/year")
        print(f"Mean: {annual_precip.mean():.0f} mm/year")
    
    return self.fields['precipitation']


## Temperature

In [ ]:
#| export
@patch
def compute_distance_from_coast(self: Terrain):
    """Calculate distance from coast for each hex using BFS."""
    
    
    n_hexes = len(self.elevations)
    distances = np.full(n_hexes, np.inf)
    
    # Find all coastal hexes (at or below sea level)
    coastal = [i for i in range(n_hexes) if self.elevations[i] <= 0]
    
    # BFS to calculate distances
    queue = deque()
    for coast_idx in coastal:
        distances[coast_idx] = 0
        queue.append((coast_idx, 0))
    
    visited = set(coastal)
    
    while queue:
        current_idx, dist = queue.popleft()
        
        # Check all neighbors
        neighbors = self.ring(current_idx, 1)
        for neighbor in neighbors:
            if neighbor >= 0 and neighbor not in visited:
                distances[neighbor] = dist + 1
                visited.add(neighbor)
                queue.append((neighbor, dist + 1))
    
    self.fields['distance_to_coast'] = distances.astype(float)
    return self.fields['distance_to_coast']



In [ ]:
#| export
@patch
def compute_temperature(self: Terrain,
                        base_temp_south=20.0,    # Southern coast 
                        base_temp_north=14.0,    # Northern coast  
                        lapse_rate=6.5):         # °C per 1000m
    """
    Temperature model for California - mean annual temperature.
    Continentality affects seasonal range, not annual mean.
    """
    if 'distance_to_coast' not in self.fields:
        self.compute_distance_from_coast()
    
    # Latitude-based baseline
    if 'latitude' in self.fields and self.geo:
        lat_fraction = (self.fields['latitude'] - self.geo.lat_min) / (self.geo.lat_max - self.geo.lat_min)
        base_temp = base_temp_south + (base_temp_north - base_temp_south) * lat_fraction
    else:
        base_temp = np.full(len(self.elevations), (base_temp_south + base_temp_north) / 2)
    
    # Apply elevation lapse rate
    temp = base_temp - (self.elevations / 1000.0) * lapse_rate
    
    self.fields['temperature'] = temp
    return temp


In [ ]:
#| export
@patch
def validate_coastal_temps(self: Terrain):
    """Check that coastal areas aren't getting tundra temperatures."""
    if 'distance_to_coast' not in self.fields or 'temperature' not in self.fields:
        return "Missing required fields"
    
    # Check coastal hexes (distance <= 2)
    coastal_mask = self.fields['distance_to_coast'] <= 2
    coastal_temps = self.fields['temperature'][coastal_mask]
    coastal_elevs = self.elevations[coastal_mask]
    
    print("=== COASTAL VALIDATION ===")
    print(f"Coastal hexes: {coastal_mask.sum()}")
    print(f"Temp range: {coastal_temps.min():.1f}°C to {coastal_temps.max():.1f}°C")
    print(f"Elevation range: {coastal_elevs.min():.0f}m to {coastal_elevs.max():.0f}m")
    print(f"Mean temp: {coastal_temps.mean():.1f}°C")
    
    # Check for problematic cold coastal areas
    cold_coastal = (coastal_mask) & (self.fields['temperature'] < 10)
    if cold_coastal.sum() > 0:
        print(f"\n⚠️  {cold_coastal.sum()} coastal hexes < 10°C")
        cold_elevs = self.elevations[cold_coastal]
        print(f"   These are at elevations: {cold_elevs.min():.0f}m to {cold_elevs.max():.0f}m")
        print(f"   (Likely coastal mountains - OK for biomes)")
    else:
        print(f"\n✓ No unrealistic cold coastal areas")
    
    # Check high elevation inland
    high_inland = (self.fields['distance_to_coast'] > 5) & (self.elevations > 2000)
    if high_inland.sum() > 0:
        high_temps = self.fields['temperature'][high_inland]
        print(f"\nHigh mountains (>2000m, inland):")
        print(f"  Count: {high_inland.sum()}")
        print(f"  Temp range: {high_temps.min():.1f}°C to {high_temps.max():.1f}°C")


In [ ]:
#| export


@patch
def climate_readiness(self: Terrain):
    """Check what fields are available for climate classification."""
    required = ['elevation', 'temperature', 'precipitation']
    optional = ['latitude', 'longitude', 'distance_to_coast']
    
    print("=== CLIMATE READINESS ===")
    print("\nRequired fields:")
    print(f"  elevation:     {'✓' if len(self.elevations) > 0 else '✗'}")
    print(f"  temperature:   {'✓' if 'temperature' in self.fields else '✗'}")
    print(f"  precipitation: {'✓' if 'precipitation' in self.fields else '✗'}")
    
    print("\nOptional fields:")
    for field in optional:
        print(f"  {field:20s} {'✓' if field in self.fields else '✗'}")
    
    print("\nAvailable fields:")
    for field in sorted(self.fields.keys()):
        print(f"  - {field}")



In [ ]:
#| export
@patch
def shrinkWeather(self: Terrain, scale=0.5,sample_radius=1):
    """Downsample terrain with all weather data preserved.
    
    This intelligently handles:
    - Elevation (max to preserve peaks)
    - Temperature (weighted average)
    - Precipitation (weighted average)
    - Distance to coast (min - closest coast matters)
    - Geographic coordinates (weighted average)
    
    Args:
        scale: Fraction of original size (0.5 = half size, 0.33 = third size)
    """
    
    # Helper to get hex pattern for sampling
    def hexes_in_range(n):
        """Finding all hexes within distance N from origin"""
        results = []
        for q in range(-n, n + 1):
            for r in range(max(-n, -q - n), min(n, -q + n) + 1):
                s = -q - r
                results.append(HexPosition(q, r, s))
        return results
    
    ring_pattern = hexes_in_range(sample_radius)
    
    # Start with elevation using max (preserve peaks)
    new_elevations, nRows, nCols = self.convolution(
        self.elevations, ring_pattern, scale, method='max')
    
    # Create new terrain with downsampled grid
    grid = HexGrid(
        nRows=nRows,
        nCols=nCols,
        radius=self.hexGrid.radius,
        style=self.hexGrid.style
    )
    
    mySize = MapSize(400, 400)  # These get dropped
    myBounds = MapRect(MapCord(0, 0), mySize)
    new_terrain = Terrain(myBounds, radius=15)
    new_terrain.hexGrid = grid
    new_terrain.elevations = new_elevations
    
    # Copy style properties
    new_terrain.colorLevels = self.colorLevels.copy() if self.colorLevels else None
    new_terrain.elevationDelta = self.elevationDelta
    new_terrain.seaLevel = self.seaLevel
    new_terrain.climate = self.climate
    
    # Downsample all fields with appropriate methods
    field_methods = {
        'temperature': 'weighted_avg',
        'precipitation': 'weighted_avg',
        'precip_rate_mmh': 'weighted_avg',
        'distance_to_coast': 'min',  # Closest coast matters
        'latitude': 'weighted_avg',
        'longitude': 'weighted_avg',
        'temp_range': 'weighted_avg'
    }
    
    for field_name, method in field_methods.items():
        if field_name in self.fields:
            downsampled, _, _ = self.convolution(
                self.fields[field_name], ring_pattern, scale, method=method)
            new_terrain.fields[field_name] = downsampled
    
    # Add color styles
    if new_terrain.colorLevels:
        for color in new_terrain.colorLevels:
            new_terrain.hexGrid.builder.add_style(color)
    
    
    return new_terrain


### Using Presets

In [ ]:
#| export
@patch
def weatherPatterns(self:TerrainPatterns):
    return {
    'tropical': ClimatePreset(
        name='Tropical',
        lat_range=(0, 15),
        base_temp_range=(26, 28),
        wind_speed=8.0,
        wind_dir=90.0,
        precip_base=0.2,
        nm=0.008,
        hw=2500.0,
        cw=0.004,
        conv_time=1000.0,
        fall_time=1000.0,
        lapse_rate=5.5,
        precip_bins=(1500, 2500, 4000, 6000),  # High rainfall expected
        description="Hot, humid, high rainfall. Trade winds from east."
    ),
    
    'subtropical': ClimatePreset(
        name='Subtropical',
        lat_range=(15, 30),
        base_temp_range=(20, 24),
        wind_speed=10.0,
        wind_dir=270.0,
        precip_base=0.12,
        nm=0.009,
        hw=2200.0,
        cw=0.003,
        conv_time=1200.0,
        fall_time=1200.0,
        lapse_rate=6.0,
        precip_bins=(600, 1000, 1800, 3000),  # Moderate
        description="Warm, moderate rainfall. Mediterranean-like."
    ),
    
    'mediterranean': ClimatePreset(
        name='Mediterranean',
        lat_range=(30, 45),
        base_temp_range=(14, 18),
        wind_speed=12.0,
        wind_dir=270.0,
        precip_base=0.08,
        nm=0.01,
        hw=2000.0,
        cw=0.0025,
        conv_time=1500.0,
        fall_time=1500.0,
        lapse_rate=6.5,
        precip_bins=(250, 500, 900, 1400),  # Drier
        description="Mild winters, dry summers. California-like."
    ),
    
    'temperate': ClimatePreset(
        name='Temperate',
        lat_range=(35, 55),
        base_temp_range=(8, 14),
        wind_speed=15.0,
        wind_dir=270.0,
        precip_base=0.15,
        nm=0.012,
        hw=1800.0,
        cw=0.003,
        conv_time=1500.0,
        fall_time=1500.0,
        lapse_rate=6.5,
        precip_bins=(800, 1400, 2200, 3500),  # Wet
        description="Four seasons, moderate rainfall. Pacific Northwest-like."
    ),
    
    'continental': ClimatePreset(
        name='Continental',
        lat_range=(40, 60),
        base_temp_range=(2, 10),
        wind_speed=12.0,
        wind_dir=270.0,
        precip_base=0.08,
        nm=0.013,
        hw=1600.0,
        cw=0.002,
        conv_time=2000.0,
        fall_time=2000.0,
        lapse_rate=7.0,
        precip_bins=(200, 400, 700, 1200),  # Dry interior
        description="Hot summers, cold winters, low rainfall. Central Asia-like."
    ),
    
    'subarctic': ClimatePreset(
        name='Subarctic',
        lat_range=(55, 70),
        base_temp_range=(-5, 5),
        wind_speed=10.0,
        wind_dir=270.0,
        precip_base=0.05,
        nm=0.015,
        hw=1400.0,
        cw=0.0015,
        conv_time=2500.0,
        fall_time=2500.0,
        lapse_rate=7.5,
        precip_bins=(150, 300, 500, 800),  # Low precipitation
        description="Long cold winters, short cool summers. Alaska-like."
    ),
    
    'arctic': ClimatePreset(
        name='Arctic',
        lat_range=(70, 85),
        base_temp_range=(-15, -5),
        wind_speed=8.0,
        wind_dir=0.0,
        precip_base=0.02,
        nm=0.018,
        hw=1200.0,
        cw=0.001,
        conv_time=3000.0,
        fall_time=3000.0,
        lapse_rate=8.0,
        precip_bins=(50, 100, 200, 400),  # Very dry
        description="Extremely cold, very low precipitation. Ice cap climate."
    ),
    
    'monsoon': ClimatePreset(
        name='Monsoon',
        lat_range=(10, 25),
        base_temp_range=(24, 28),
        wind_speed=12.0,
        wind_dir=135.0,
        precip_base=0.25,
        nm=0.007,
        hw=2800.0,
        cw=0.005,
        conv_time=800.0,
        fall_time=800.0,
        lapse_rate=5.5,
        precip_bins=(2000, 3500, 5500, 8000),  # Extreme rainfall
        description="Extreme seasonal rainfall. India/Southeast Asia-like."
    ),
    
    'oceanic': ClimatePreset(
        name='Oceanic',
        lat_range=(40, 60),
        base_temp_range=(8, 12),
        wind_speed=18.0,
        wind_dir=270.0,
        precip_base=0.2,
        nm=0.01,
        hw=2000.0,
        cw=0.004,
        conv_time=1000.0,
        fall_time=1000.0,
        lapse_rate=6.0,
        precip_bins=(1000, 1800, 2800, 4000),  # Very wet
        description="Mild, wet year-round. British Isles-like."
    ),
}

In [ ]:
#| export
@patch
def compute_weather(self:Terrain, 
    force_recompute: bool = False, #don't recompute when you don't have to
    debug=False):
    """
    Apply the stored climate preset to compute precipitation, temperature, and climate zones.
    
    Args:
        terrain: Terrain with climate_preset set
        force_recompute: Recompute even if fields already exist
    """
    terrain = self
    preset = terrain.climate
   

    # Compute precipitation
    if 'precipitation' not in terrain.fields or force_recompute:
        if debug:
            print("\nComputing precipitation...")
        terrain.compute_precipitation_sb(
            wind_speed=preset.wind_speed,
            wind_dir=preset.wind_dir,
            precip_base=preset.precip_base,
            nm=preset.nm,
            hw=preset.hw,
            cw=preset.cw,
            conv_time=preset.conv_time,
            fall_time=preset.fall_time,
            debug = debug
        )
    
    # Compute distance to coast
    if 'distance_to_coast' not in terrain.fields or force_recompute:
        if debug:
            print("Computing distance to coast...")
        terrain.compute_distance_from_coast()
    
    # Compute temperature
    if 'temperature' not in terrain.fields or force_recompute:
        if debug:
            print("Computing temperature...")
        terrain.compute_temperature(
            base_temp_south=preset.base_temp_range[0],
            base_temp_north=preset.base_temp_range[1],
            lapse_rate=preset.lapse_rate
        )



### Examples

In [ ]:
#| export
@patch
def bayArea_map(self:TerraDemo):
    """Load Maui terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/BayArea.txt').open() as f:
        return Terrain.decode(f.read())


In [ ]:
terr = TerraDemo().bayArea_map()
terr.geo, terr.climate

(GeoBounds(lat_min=37.57, lat_max=38.03, lon_min=-122.69, lon_max=-121.97),
 ClimatePreset(name='Mediterranean', lat_range=(30.0, 45.0), base_temp_range=(14.0, 18.0), wind_speed=12.0, wind_dir=270.0, precip_base=0.08, nm=0.01, hw=2000.0, cw=0.0025, conv_time=1500.0, fall_time=1500.0, lapse_rate=6.5, precip_bins=(400.0, 800.0, 1500.0, 2500.0), description='Mild winters, dry summers. California-like.'))

In [ ]:
#| export
@patch
def aussie_map(self:TerraDemo):
    """Load Maui terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/australia.txt').open() as f:
        return Terrain.decode(f.read())

In [ ]:
aus = TerraDemo().aussie_map()
aus.hexGrid.adjustRadius(8)
aus.colorMap()
ausGrid = aus.hexGrid
ausGrid.update()

#ausGrid.builder.adjust("regions", ausGrid.styleLayerOrdered(styles=aus.colorLevels))
ausGrid.builder.show()

In [ ]:
#| export
@patch
def california_map(self: TerraDemo):
    """Load California terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/ca_nv_elevation_grid.txt').open() as f:
        return Terrain.decode(f.read())
    
    # Set the geographic bounds that match your data
    terrain_ca.geo = GeoBounds(
        lat_min=32.5,   # Southern California/Nevada border
        lat_max=42.0,   # Northern California/Oregon border
        lon_min=-124.5, # Pacific coast
        lon_max=-114.5  # Eastern Nevada
    )
    
    # Compute hex coordinates
    terrain_ca._compute_hex_coordinates()
    terrain_ca.climate = ClimatePreset(
    name='California',
    lat_range=(32.5, 42.0),  # Your actual CA bounds
    base_temp_range=(20, 14),  # Warmer south, cooler north
    wind_speed=10.0,
    wind_dir=270.0,  # Pacific winds from west
    precip_base=0.1,
    nm=0.01,
    hw=2000.0,
    cw=0.002,
    conv_time=2000.0,
    fall_time=2000.0,
    lapse_rate=6.5,
    precip_bins=(200, 500, 1000, 2000),  # Desert to rainforest range
    description="Mediterranean with extreme orographic effects. Mojave to redwoods."
)
    terrain_ca.hexGrid.adjustRadius(30)
    with open('../HexMagic/data/templates/ca_nv_elevation_grid.txt', 'w') as f:
        f.write(terrain_ca.encode())
    
    return terrain_ca

In [ ]:
terr = TerraDemo().california_map()
terr.geo, terr.climate

(GeoBounds(lat_min=32.5, lat_max=42.0, lon_min=-124.5, lon_max=-114.5),
 ClimatePreset(name='California', lat_range=(32.5, 42.0), base_temp_range=(20.0, 14.0), wind_speed=10.0, wind_dir=270.0, precip_base=0.1, nm=0.01, hw=2000.0, cw=0.002, conv_time=2000.0, fall_time=2000.0, lapse_rate=6.5, precip_bins=(200.0, 500.0, 1000.0, 2000.0), description='Mediterranean with extreme orographic effects. Mojave to redwoods.'))

In [ ]:
#| export
@patch
def maui_map(self:TerraDemo):
    """Load Maui terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/maui.txt').open() as f:
        return Terrain.decode(f.read())
    
    # Set Maui's geographic bounds
    terrain.geo = GeoBounds(
        lat_min=20.57,   # Southern tip (near Makena)
        lat_max=21.03,   # Northern tip (near Kahakuloa)
        lon_min=-156.69, # Western tip (West Maui)
        lon_max=-155.97  # Eastern tip (Haleakalā/Hāna)
    )
    
    # Compute hex coordinates
    terrain._compute_hex_coordinates()
    
    # Maui-specific precipitation model
    # Trade winds from northeast at ~50-60 degrees
    terrain.climate = ClimatePreset(
    name='Maui',
    lat_range=(20.57, 21.03),  # Your actual CA bounds
    base_temp_range=(26, 28),
    wind_speed=8.0,        # Slightly stronger trade winds
    wind_dir=50.0,         
    precip_base=0.15,      # MUCH higher base moisture (tropical ocean)
    nm=0.008,              # Less stable (more convection)
    hw=2500.0,             # Higher moisture scale height
    cw=0.003,              # MUCH stronger orographic effect
    conv_time=1000.0,      # Faster conversion
    fall_time=1000.0 
    )

    with open('../HexMagic/data/templates/maui.txt', 'w') as f:
        f.write(terrain.encode())


    return terrain




In [ ]:
terr = TerraDemo().maui_map()
terr.geo, terr.climate

(GeoBounds(lat_min=20.57, lat_max=21.03, lon_min=-156.69, lon_max=-155.97),
 ClimatePreset(name='Maui', lat_range=(20.57, 21.03), base_temp_range=(26, 28), wind_speed=8.0, wind_dir=50.0, precip_base=0.15, nm=0.008, hw=2500.0, cw=0.003, conv_time=1000.0, fall_time=1000.0, lapse_rate=6.5, precip_bins=(400, 800, 1500, 2500), description=''))

In [ ]:
#| export
@patch
def validate_coastal_temps(self: Terrain):
    """Check that coastal areas aren't getting tundra temperatures."""
    if 'distance_to_coast' not in self.fields or 'temperature' not in self.fields:
        return "Missing required fields"
    
    # Check coastal hexes (distance <= 2)
    coastal_mask = self.fields['distance_to_coast'] <= 2
    coastal_temps = self.fields['temperature'][coastal_mask]
    coastal_elevs = self.elevations[coastal_mask]
    
    print("=== COASTAL VALIDATION ===")
    print(f"Coastal hexes: {coastal_mask.sum()}")
    print(f"Temp range: {coastal_temps.min():.1f}°C to {coastal_temps.max():.1f}°C")
    print(f"Elevation range: {coastal_elevs.min():.0f}m to {coastal_elevs.max():.0f}m")
    print(f"Mean temp: {coastal_temps.mean():.1f}°C")
    
    # Check for problematic cold coastal areas
    cold_coastal = (coastal_mask) & (self.fields['temperature'] < 10)
    if cold_coastal.sum() > 0:
        print(f"\n⚠️  {cold_coastal.sum()} coastal hexes < 10°C")
        cold_elevs = self.elevations[cold_coastal]
        print(f"   These are at elevations: {cold_elevs.min():.0f}m to {cold_elevs.max():.0f}m")
        print(f"   (Likely coastal mountains - OK for biomes)")
    else:
        print(f"\n✓ No unrealistic cold coastal areas")
    
    # Check high elevation inland
    high_inland = (self.fields['distance_to_coast'] > 5) & (self.elevations > 2000)
    if high_inland.sum() > 0:
        high_temps = self.fields['temperature'][high_inland]
        print(f"\nHigh mountains (>2000m, inland):")
        print(f"  Count: {high_inland.sum()}")
        print(f"  Temp range: {high_temps.min():.1f}°C to {high_temps.max():.1f}°C")


In [ ]:
#| export
@patch
def sanFran_map(self: TerraDemo):
    """Load San Francisco terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/san_francisco.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    # San Francisco area bounds
    terrain.geo = GeoBounds(
        lat_min=37.70,
        lat_max=37.85,
        lon_min=-122.52,
        lon_max=-122.35
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='San Francisco',
        lat_range=(37, 38),
        base_temp_range=(14, 13),  # Famously cool and stable
        wind_speed=15.0,
        wind_dir=270.0,  # Westerlies off Pacific
        precip_base=0.06,
        nm=0.01,
        hw=2000.0,
        cw=0.002,
        conv_time=1500.0,
        fall_time=1500.0,
        lapse_rate=6.5,
        precip_bins=(300, 500, 800, 1200),
        description="Cool Mediterranean. Famous fog from cold Pacific upwelling."
    )
    
    return terrain

@patch
def agincourt_map(self: TerraDemo):
    """Load Agincourt terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/agincourt.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    # Agincourt is in Pas-de-Calais, northern France
    terrain.geo = GeoBounds(
        lat_min=50.40,
        lat_max=50.55,
        lon_min=2.05,
        lon_max=2.25
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Oceanic Northern France',
        lat_range=(50, 51),
        base_temp_range=(10, 11),  # Mild oceanic
        wind_speed=12.0,
        wind_dir=250.0,  # Prevailing SW winds from Atlantic
        precip_base=0.12,
        nm=0.012,
        hw=1800.0,
        cw=0.003,
        conv_time=1200.0,
        fall_time=1200.0,
        lapse_rate=6.0,
        precip_bins=(600, 800, 1000, 1400),
        description="Oceanic climate. Wet, mild year-round. Atlantic influence."
    )
    
    return terrain


In [ ]:
#| export
@patch
def pompeii_map(self: TerraDemo):
    """Load Pompeii terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/pompeii.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    # Pompeii area - near Naples, includes Vesuvius
    terrain.geo = GeoBounds(
        lat_min=40.70,
        lat_max=40.85,
        lon_min=14.40,
        lon_max=14.55
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Mediterranean Italy',
        lat_range=(40, 41),
        base_temp_range=(16, 15),  # Warm Mediterranean
        wind_speed=8.0,
        wind_dir=240.0,  # SW winds from Tyrrhenian Sea
        precip_base=0.10,
        nm=0.01,
        hw=2000.0,
        cw=0.003,
        conv_time=1500.0,
        fall_time=1500.0,
        lapse_rate=6.5,
        precip_bins=(400, 700, 1000, 1400),
        description="Classic Mediterranean. Hot dry summers, mild wet winters."
    )
    
    return terrain

@patch
def iwo_jima_map(self: TerraDemo):
    """Load Iwo Jima terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/iwo_jima.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    # Iwo Jima - small volcanic island in Pacific
    terrain.geo = GeoBounds(
        lat_min=24.73,
        lat_max=24.80,
        lon_min=141.28,
        lon_max=141.35
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Subtropical Pacific',
        lat_range=(24, 25),
        base_temp_range=(23, 24),  # Warm year-round
        wind_speed=10.0,
        wind_dir=70.0,  # Trade winds from ENE
        precip_base=0.15,
        nm=0.008,
        hw=2400.0,
        cw=0.004,
        conv_time=1000.0,
        fall_time=1000.0,
        lapse_rate=5.5,
        precip_bins=(800, 1200, 1800, 2500),
        description="Subtropical oceanic. Warm, humid. Typhoon season June-Oct."
    )
    
    return terrain

@patch
def normandy_map(self: TerraDemo):
    """Load Normandy terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/normandy.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    # Normandy beaches area
    terrain.geo = GeoBounds(
        lat_min=49.30,
        lat_max=49.45,
        lon_min=-0.95,
        lon_max=-0.75
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Oceanic Normandy',
        lat_range=(49, 50),
        base_temp_range=(11, 10),  # Cool oceanic
        wind_speed=14.0,
        wind_dir=260.0,  # Prevailing westerlies from Atlantic
        precip_base=0.14,
        nm=0.012,
        hw=1800.0,
        cw=0.003,
        conv_time=1200.0,
        fall_time=1200.0,
        lapse_rate=6.0,
        precip_bins=(650, 850, 1100, 1500),
        description="Oceanic. Mild, wet year-round. Strong Atlantic influence."
    )
    
    return terrain

@patch
def grand_canyon_map(self: TerraDemo):
    """Load Grand Canyon terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/grand_canyon.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    # Grand Canyon area
    terrain.geo = GeoBounds(
        lat_min=35.95,
        lat_max=36.25,
        lon_min=-112.30,
        lon_max=-111.90
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='High Desert Arizona',
        lat_range=(35, 37),
        base_temp_range=(14, 12),  # Varies hugely with elevation
        wind_speed=8.0,
        wind_dir=240.0,  # SW monsoon influence in summer
        precip_base=0.03,  # Very dry
        nm=0.015,
        hw=1500.0,
        cw=0.002,
        conv_time=2000.0,
        fall_time=2000.0,
        lapse_rate=7.0,  # Steeper due to dry air
        precip_bins=(150, 300, 450, 700),  # Desert precipitation
        description="High desert. Extreme elevation range (rim to river ~1500m). Hot summers, cold winters at rim."
    )
    
    return terrain


In [ ]:
#| export
@patch
def sydney_map(self: TerraDemo):
    """Load Sydney terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/sydney.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    terrain.geo = GeoBounds(
        lat_min=-34.05,
        lat_max=-33.70,
        lon_min=150.95,
        lon_max=151.35
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Humid Subtropical Sydney',
        lat_range=(-34, -33),
        base_temp_range=(18, 17),  # Mild year-round
        wind_speed=10.0,
        wind_dir=90.0,  # Easterlies from Pacific
        precip_base=0.12,
        nm=0.01,
        hw=2200.0,
        cw=0.003,
        conv_time=1200.0,
        fall_time=1200.0,
        lapse_rate=6.0,
        precip_bins=(600, 900, 1200, 1600),
        description="Humid subtropical. Warm summers, mild winters. Rain year-round."
    )
    
    return terrain

@patch
def hong_kong_map(self: TerraDemo):
    """Load Hong Kong terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/hong_kong.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    terrain.geo = GeoBounds(
        lat_min=22.15,
        lat_max=22.55,
        lon_min=113.85,
        lon_max=114.40
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Subtropical Monsoon Hong Kong',
        lat_range=(22, 23),
        base_temp_range=(23, 22),  # Hot and humid
        wind_speed=10.0,
        wind_dir=135.0,  # SE monsoon in summer
        precip_base=0.18,
        nm=0.008,
        hw=2600.0,
        cw=0.004,
        conv_time=900.0,
        fall_time=900.0,
        lapse_rate=5.5,
        precip_bins=(1200, 1800, 2400, 3200),
        description="Subtropical monsoon. Hot wet summers, mild dry winters. Typhoon season."
    )
    
    return terrain


In [ ]:
aus = TerraDemo().sydney_map()
aus.hexGrid.adjustRadius(8)
aus.elevationDelta = 40
aus.colorMap()
ausGrid = aus.hexGrid
ausGrid.update()

#ausGrid.builder.adjust("regions", ausGrid.styleLayerOrdered(styles=aus.colorLevels))
ausGrid.builder.show()

In [ ]:
#| export
@patch
def reykjavik_map(self: TerraDemo):
    """Load Reykjavik terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/reykjavik.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    terrain.geo = GeoBounds(
        lat_min=63.95,
        lat_max=64.20,
        lon_min=-22.10,
        lon_max=-21.70
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Subarctic Oceanic Iceland',
        lat_range=(63, 65),
        base_temp_range=(5, 4),  # Cool year-round, Gulf Stream moderates
        wind_speed=14.0,
        wind_dir=240.0,  # SW winds from Atlantic
        precip_base=0.10,
        nm=0.013,
        hw=1600.0,
        cw=0.003,
        conv_time=1800.0,
        fall_time=1800.0,
        lapse_rate=6.5,
        precip_bins=(400, 700, 1000, 1400),
        description="Subarctic oceanic. Mild for latitude due to Gulf Stream. Windy, changeable."
    )
    
    return terrain

@patch
def rio_map(self: TerraDemo):
    """Load Rio de Janeiro terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/rio.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    terrain.geo = GeoBounds(
        lat_min=-23.10,
        lat_max=-22.75,
        lon_min=-43.55,
        lon_max=-43.10
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Tropical Rio',
        lat_range=(-23, -22),
        base_temp_range=(25, 26),  # Hot year-round
        wind_speed=8.0,
        wind_dir=90.0,  # Trade winds from east/Atlantic
        precip_base=0.15,
        nm=0.007,
        hw=2600.0,
        cw=0.004,
        conv_time=900.0,
        fall_time=900.0,
        lapse_rate=5.5,
        precip_bins=(1000, 1400, 1800, 2500),
        description="Tropical savanna. Hot humid summers, warm dry winters. Orographic rain on mountains."
    )
    
    return terrain

@patch
def santorini_map(self: TerraDemo):
    """Load Santorini terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/santorini.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    terrain.geo = GeoBounds(
        lat_min=36.35,
        lat_max=36.48,
        lon_min=25.35,
        lon_max=25.50
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Mediterranean Aegean',
        lat_range=(36, 37),
        base_temp_range=(18, 17),  # Warm Mediterranean
        wind_speed=12.0,
        wind_dir=350.0,  # Etesian/Meltemi winds from north in summer
        precip_base=0.04,  # Very dry
        nm=0.012,
        hw=1800.0,
        cw=0.002,
        conv_time=1800.0,
        fall_time=1800.0,
        lapse_rate=6.5,
        precip_bins=(200, 350, 500, 700),
        description="Hot-summer Mediterranean. Very dry summers, mild wet winters. Strong summer winds."
    )
    
    return terrain

@patch
def kathmandu_map(self: TerraDemo):
    """Load Kathmandu terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/kathmandu.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    terrain.geo = GeoBounds(
        lat_min=27.60,
        lat_max=27.80,
        lon_min=85.25,
        lon_max=85.45
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Subtropical Highland Monsoon',
        lat_range=(27, 28),
        base_temp_range=(20, 18),  # Moderate due to elevation (~1400m)
        wind_speed=6.0,
        wind_dir=180.0,  # Monsoon from south in summer
        precip_base=0.12,
        nm=0.008,
        hw=2400.0,
        cw=0.005,  # Strong orographic effect - Himalayas!
        conv_time=800.0,
        fall_time=800.0,
        lapse_rate=6.0,
        precip_bins=(800, 1200, 1800, 2500),
        description="Subtropical highland with monsoon. Warm wet summers, cool dry winters. Extreme orographic effects."
    )
    
    return terrain


NameError: name 'patch' is not defined

In [ ]:
ls ../HexMagic/data/templates

BayArea.txt               grand_canyon.txt  normandy.txt       santorini.txt
agincourt.txt             hong_kong.txt     pompeii.txt        sydney.txt
australia.txt             iwo_jima.txt      reykjavik.txt      three_gorges.txt
ca_nv_elevation_grid.txt  kathmandu.txt     rio.txt            vancouver.txt
cape_town.txt             maui.txt          san_francisco.txt


Which of these maps is going to have the largest range of weather

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

**California** stands out as the winner for weather range, based on what's encoded in these maps:

- **Latitude span**: 9.5° (32.5° to 42.0°) — largest of the regional maps
- **Elevation extremes**: Death Valley (below sea level) to Sierra Nevada peaks (4000m+)
- **Precipitation range**: The preset notes "Desert to rainforest range" — Mojave gets ~100mm/year while the redwood coast gets 2500mm+
- **Temperature**: Combines hot desert, cool foggy coast, and alpine conditions

The description literally says: *"Mediterranean with extreme orographic effects. Mojave to redwoods."*

**Australia** could rival this if it covers the whole continent (tropical north, desert interior, temperate south), but I'd need to check its actual bounds in the file.

Want to test this empirically? You could compute weather on both and compare the ranges:

```python
terr_ca = TerraDemo().california_map()
terr_ca.compute_weather()
print(f"CA Precip range: {terr_ca.fields['precipitation'].min():.0f} - {terr_ca.fields['precipitation'].max():.0f} mm")
print(f"CA Temp range: {terr_ca.fields['temperature'].min():.1f} - {terr_ca.fields['temperature'].max():.1f} °C")
```

can you make maps for reykjavik,rio, santorini, and kathmandu

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Sure! Here are the four map loaders with appropriate geographic bounds and climate presets:

```python
@patch
def reykjavik_map(self: TerraDemo):
    """Load Reykjavik terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/reykjavik.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    terrain.geo = GeoBounds(
        lat_min=63.95,
        lat_max=64.20,
        lon_min=-22.10,
        lon_max=-21.70
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Subarctic Oceanic Iceland',
        lat_range=(63, 65),
        base_temp_range=(5, 4),  # Cool year-round, Gulf Stream moderates
        wind_speed=14.0,
        wind_dir=240.0,  # SW winds from Atlantic
        precip_base=0.10,
        nm=0.013,
        hw=1600.0,
        cw=0.003,
        conv_time=1800.0,
        fall_time=1800.0,
        lapse_rate=6.5,
        precip_bins=(400, 700, 1000, 1400),
        description="Subarctic oceanic. Mild for latitude due to Gulf Stream. Windy, changeable."
    )
    
    return terrain

@patch
def rio_map(self: TerraDemo):
    """Load Rio de Janeiro terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/rio.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    terrain.geo = GeoBounds(
        lat_min=-23.10,
        lat_max=-22.75,
        lon_min=-43.55,
        lon_max=-43.10
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Tropical Rio',
        lat_range=(-23, -22),
        base_temp_range=(25, 26),  # Hot year-round
        wind_speed=8.0,
        wind_dir=90.0,  # Trade winds from east/Atlantic
        precip_base=0.15,
        nm=0.007,
        hw=2600.0,
        cw=0.004,
        conv_time=900.0,
        fall_time=900.0,
        lapse_rate=5.5,
        precip_bins=(1000, 1400, 1800, 2500),
        description="Tropical savanna. Hot humid summers, warm dry winters. Orographic rain on mountains."
    )
    
    return terrain

@patch
def santorini_map(self: TerraDemo):
    """Load Santorini terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/santorini.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    terrain.geo = GeoBounds(
        lat_min=36.35,
        lat_max=36.48,
        lon_min=25.35,
        lon_max=25.50
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Mediterranean Aegean',
        lat_range=(36, 37),
        base_temp_range=(18, 17),  # Warm Mediterranean
        wind_speed=12.0,
        wind_dir=350.0,  # Etesian/Meltemi winds from north in summer
        precip_base=0.04,  # Very dry
        nm=0.012,
        hw=1800.0,
        cw=0.002,
        conv_time=1800.0,
        fall_time=1800.0,
        lapse_rate=6.5,
        precip_bins=(200, 350, 500, 700),
        description="Hot-summer Mediterranean. Very dry summers, mild wet winters. Strong summer winds."
    )
    
    return terrain

@patch
def kathmandu_map(self: TerraDemo):
    """Load Kathmandu terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/kathmandu.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    terrain.geo = GeoBounds(
        lat_min=27.60,
        lat_max=27.80,
        lon_min=85.25,
        lon_max=85.45
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Subtropical Highland Monsoon',
        lat_range=(27, 28),
        base_temp_range=(20, 18),  # Moderate due to elevation (~1400m)
        wind_speed=6.0,
        wind_dir=180.0,  # Monsoon from south in summer
        precip_base=0.12,
        nm=0.008,
        hw=2400.0,
        cw=0.005,  # Strong orographic effect - Himalayas!
        conv_time=800.0,
        fall_time=800.0,
        lapse_rate=6.0,
        precip_bins=(800, 1200, 1800, 2500),
        description="Subtropical highland with monsoon. Warm wet summers, cool dry winters. Extreme orographic effects."
    )
    
    return terrain
```

A few notes on the climate choices:

- **Reykjavik**: Subarctic but surprisingly mild (Gulf Stream!), very windy
- **Rio**: Tropical with dramatic coastal mountains causing orographic rain
- **Santorini**: Dry Mediterranean, famous Meltemi winds from the north
- **Kathmandu**: Valley at ~1400m elevation, massive monsoon influence from Himalayas

Want me to adjust any of the bounds or climate parameters? You may need to tweak them based on the actual extent of your template files.

can you build the climate ones for san franisco and againcort with a proper climate and using the resources

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

```python
@patch
def sanFran_map(self: TerraDemo):
    """Load San Francisco terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/san_francisco.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    # San Francisco area bounds
    terrain.geo = GeoBounds(
        lat_min=37.70,
        lat_max=37.85,
        lon_min=-122.52,
        lon_max=-122.35
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='San Francisco',
        lat_range=(37, 38),
        base_temp_range=(14, 13),  # Famously cool and stable
        wind_speed=15.0,
        wind_dir=270.0,  # Westerlies off Pacific
        precip_base=0.06,
        nm=0.01,
        hw=2000.0,
        cw=0.002,
        conv_time=1500.0,
        fall_time=1500.0,
        lapse_rate=6.5,
        precip_bins=(300, 500, 800, 1200),
        description="Cool Mediterranean. Famous fog from cold Pacific upwelling."
    )
    
    return terrain

@patch
def agincourt_map(self: TerraDemo):
    """Load Agincourt terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/agincourt.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    # Agincourt is in Pas-de-Calais, northern France
    terrain.geo = GeoBounds(
        lat_min=50.40,
        lat_max=50.55,
        lon_min=2.05,
        lon_max=2.25
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Oceanic Northern France',
        lat_range=(50, 51),
        base_temp_range=(10, 11),  # Mild oceanic
        wind_speed=12.0,
        wind_dir=250.0,  # Prevailing SW winds from Atlantic
        precip_base=0.12,
        nm=0.012,
        hw=1800.0,
        cw=0.003,
        conv_time=1200.0,
        fall_time=1200.0,
        lapse_rate=6.0,
        precip_bins=(600, 800, 1000, 1400),
        description="Oceanic climate. Wet, mild year-round. Atlantic influence."
    )
    
    return terrain
```

Note: Make sure the files exist at those paths in your HexMagic package. If they're in your local `data/templates/` folder instead, you'd need to either:
1. Move them into the package, or
2. Use the simpler `open()` approach but with the full path

and now for pompeii.txt iwo_jima.txt normandy.txt grand_canyon.txt

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

```python
@patch
def pompeii_map(self: TerraDemo):
    """Load Pompeii terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/pompeii.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    # Pompeii area - near Naples, includes Vesuvius
    terrain.geo = GeoBounds(
        lat_min=40.70,
        lat_max=40.85,
        lon_min=14.40,
        lon_max=14.55
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Mediterranean Italy',
        lat_range=(40, 41),
        base_temp_range=(16, 15),  # Warm Mediterranean
        wind_speed=8.0,
        wind_dir=240.0,  # SW winds from Tyrrhenian Sea
        precip_base=0.10,
        nm=0.01,
        hw=2000.0,
        cw=0.003,
        conv_time=1500.0,
        fall_time=1500.0,
        lapse_rate=6.5,
        precip_bins=(400, 700, 1000, 1400),
        description="Classic Mediterranean. Hot dry summers, mild wet winters."
    )
    
    return terrain

@patch
def iwo_jima_map(self: TerraDemo):
    """Load Iwo Jima terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/iwo_jima.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    # Iwo Jima - small volcanic island in Pacific
    terrain.geo = GeoBounds(
        lat_min=24.73,
        lat_max=24.80,
        lon_min=141.28,
        lon_max=141.35
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Subtropical Pacific',
        lat_range=(24, 25),
        base_temp_range=(23, 24),  # Warm year-round
        wind_speed=10.0,
        wind_dir=70.0,  # Trade winds from ENE
        precip_base=0.15,
        nm=0.008,
        hw=2400.0,
        cw=0.004,
        conv_time=1000.0,
        fall_time=1000.0,
        lapse_rate=5.5,
        precip_bins=(800, 1200, 1800, 2500),
        description="Subtropical oceanic. Warm, humid. Typhoon season June-Oct."
    )
    
    return terrain

@patch
def normandy_map(self: TerraDemo):
    """Load Normandy terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/normandy.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    # Normandy beaches area
    terrain.geo = GeoBounds(
        lat_min=49.30,
        lat_max=49.45,
        lon_min=-0.95,
        lon_max=-0.75
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='Oceanic Normandy',
        lat_range=(49, 50),
        base_temp_range=(11, 10),  # Cool oceanic
        wind_speed=14.0,
        wind_dir=260.0,  # Prevailing westerlies from Atlantic
        precip_base=0.14,
        nm=0.012,
        hw=1800.0,
        cw=0.003,
        conv_time=1200.0,
        fall_time=1200.0,
        lapse_rate=6.0,
        precip_bins=(650, 850, 1100, 1500),
        description="Oceanic. Mild, wet year-round. Strong Atlantic influence."
    )
    
    return terrain

@patch
def grand_canyon_map(self: TerraDemo):
    """Load Grand Canyon terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/grand_canyon.txt').open() as f:
        terrain = Terrain.decode(f.read())
    
    # Grand Canyon area
    terrain.geo = GeoBounds(
        lat_min=35.95,
        lat_max=36.25,
        lon_min=-112.30,
        lon_max=-111.90
    )
    
    terrain._compute_hex_coordinates()
    
    terrain.climate = ClimatePreset(
        name='High Desert Arizona',
        lat_range=(35, 37),
        base_temp_range=(14, 12),  # Varies hugely with elevation
        wind_speed=8.0,
        wind_dir=240.0,  # SW monsoon influence in summer
        precip_base=0.03,  # Very dry
        nm=0.015,
        hw=1500.0,
        cw=0.002,
        conv_time=2000.0,
        fall_time=2000.0,
        lapse_rate=7.0,  # Steeper due to dry air
        precip_bins=(150, 300, 450, 700),  # Desert precipitation
        description="High desert. Extreme elevation range (rim to river ~1500m). Hot summers, cold winters at rim."
    )
    
    return terrain
```

In [ ]:
!ls ../HexGrid/data/templates/

### Drawing

In [ ]:
#| export
@patch
def visualize_precipitation(self: Terrain, layer_name="precipitation",f=None):
    """Visualize precipitation with color gradient and legend."""
    if 'precipitation' not in self.fields:
        print("Computing precipitation first...")
        self.compute_precipitation_sb()
    
    precip = self.fields['precipitation']
    
    # Precipitation thresholds (mm/year)
    thresholds = [0, 150, 300, 500, 750, 1000, 1500, 2000]
    
    # Get patterns from precipitationStyle
    patGen = TerrainPatterns(self)
    terrain = self

    terrain.colorMap()
    
    # Find ocean hexes (level 0)
    ocean_hexes = terrain.find_region_at_level(0)
    ocean_region = HexRegion(hexes=ocean_hexes, hexGrid=terrain.hexGrid)
    
    # Create wave pattern with ocean blues
   
    wave = patGen.wavePattern("ocean_waves_pat", 
                              amplitude=4, 
                              wavelength=16, 
                              color="#1565C0",      # stroke: medium blue
                              fill="#E3F2FD")       # fill: light blue

    oceanStyle = StyleCSS("ocean_waves", fill=f"url(#ocean_waves_pat)")
    self.builder.add_definition(wave)
    self.builder.add_style(oceanStyle)

    patterns, styles = patGen.precipitationStyle(0.1)
    
    # Add patterns to builder
    for p in patterns:
        self.builder.add_definition(p)

    for s in styles:
        self.builder.add_style(s)
    
    # Bin precipitation values
    precip_indices = np.digitize(precip, thresholds) - 1
    precip_indices = np.clip(precip_indices, 0, len(patterns) - 1)
    
    sgrid = self.hexGrid
   
    climateRegions = sgrid.regions_by_value(precip_indices)

    for region in climateRegions:
        idx  =  region.hexes.pop()
        region.hexes.add(idx)
        
        styleI = int(precip_indices[idx])
        style = styles[styleI]
        for i in region:
            sgrid.hexes[i].style = style

    for i in ocean_region:
        sgrid.hexes[i].style = oceanStyle

    overlay =  sgrid.styleLayerOrdered(
        styles=styles,
        f=f)


   
    sgrid.builder.adjust(layer_name,overlay)

    legend = sgrid.builder.legendOverlay(styles)
    sgrid.builder.adjust("legend", legend)
    



In [ ]:
#| export
@patch
def dotted_field(self: Terrain, field='precipitation',
                  flow_levels: int = 5,
                  min_density: float = 0.25,  # Minimum dot density for dry areas
                  color:str = "#cccccc",
                  debug: bool = False) -> str:
    """Combine climate colors with precipitation-based dot density.
    
    Args:
        flow_levels: Number of dot density levels (default 5)
        min_density: Minimum density for low-precip areas (0.25 = 25%)
        debug: Print pattern generation info
    
    Returns:
        SVG string for the overlay
    """
    
    if  field not in self.fields:
        raise ValueError(f"Need both {field} computed")
    
    
    # Generate patterns for each climate type
    patternGen = TerrainPatterns(self)
    all_patterns = []
      
        # Create flow_levels density patterns for this climate color
    patterns = patternGen.ballDensity(
        levels=flow_levels + 3,
        fills=[color],
        prefix=f"{field}_ball"
    )[:flow_levels]
        
    if debug:
        print(f"{climate_type.name}: generated {len(patterns)} patterns with color {color}")
    
    all_patterns.extend(patterns)
    
    # Build overlay
    grid = self.hexGrid
    overlay = ""
    used_patterns = set()
    
    precip_data = self.fields[field]
    
    # Find max precipitation for normalization
    max_precip = precip_data.max()
    min_precip = precip_data.min()
    
    if debug:
        print(f"\n{field} range: {min_precip:.0f} - {max_precip:.0f} mm/year")
        print(f"Min density: {min_density * 100}%")
    
    # Generate hex polygons
    for i, precip in enumerate(precip_data):
        
        # Normalize precipitation to [min_density, 1.0] range
        if max_precip > min_precip:
            normalized_precip = (precip - min_precip) / (max_precip - min_precip)  # [0, 1]
            # Scale to [min_density, 1.0]
            scaled_precip = min_density + normalized_precip * (1.0 - min_density)
            precip_level = int(scaled_precip * (flow_levels - 1))
            precip_level = min(precip_level, flow_levels - 1)
        else:
            precip_level = 0
        
        # Calculate pattern index: climate_val * flow_levels + precip_level
        pattern_idx =  precip_level
        
        # Bounds check
        if pattern_idx >= 0 and pattern_idx < len(all_patterns):
            used_patterns.add(pattern_idx)
            
            patName = all_patterns[pattern_idx].attributes['id']
            fill = f"url(#{patName})"
            
            # Generate polygon
            hex_obj = grid.hexes[i]
            ret = "<polygon points=\""
            for point in hex_obj.vertices():
                ret += f"{point.x:.0f},{point.y:.0f} "
            ret += f"\" style=\"fill:{fill}\"/>"
            
            overlay += "\t" + ret + "\n"
    
    # Add used patterns to builder
    for pattern_idx in sorted(used_patterns):
        grid.builder.add_definition(all_patterns[pattern_idx])
    
    if debug:
        print(f"\nUsed {len(used_patterns)} patterns out of {len(all_patterns)}")
    
    return overlay


In [ ]:
#| export
@patch
def add_rain_overlay(self: Terrain, layer_name="climate_precip",debug=False):
    """Combine climate colors with precipitation-based dot density."""
   
    overlay =  self.dotted_field(debug=debug)
    self.hexGrid.builder.adjust(layer_name, overlay)
    
    return self

In [ ]:
#| export
class WeatherColor:
    """Color scheme for weather map."""

    def __init__(self,color: str = "#000000"):
        self.color = color  # <-- Need the assignment!
        self.r, self.g, self.b = int(color[1:3], 16), int(color[3:5], 16), int(color[5:7], 16)
        self.h, self.l, self.s = colorsys.rgb_to_hls(self.r/255, self.g/255, self.b/255)

    def __str__(self):
        return self.color

    def __repr__(self):
        return f"r:{self.r:02} g:{self.g:02} b:{self.b:02}   "

    
    @staticmethod
    def temp_to_color(temp: float) -> 'WeatherColor':
        """Map temperature to smooth weather map gradient (purple → blue → green → yellow → red)."""
  
        
        # Define color stops: (temperature, hex_color)
        stops = [
            (-15, "#8B4789"),  # Purple (very cold)
            (0,   "#4A90E2"),  # Blue (freezing)
            (10,  "#50C878"),  # Green (cool)
            (20,  "#F4D03F"),  # Yellow (warm)
            (30,  "#E74C3C"),  # Red (hot)
        ]
        
        # Clamp temperature to range
        if temp <= stops[0][0]:
            return stops[0][1]
        if temp >= stops[-1][0]:
            return stops[-1][1]
        
        # Find the two stops to interpolate between
        for i in range(len(stops) - 1):
            temp1, color1 = stops[i]
            temp2, color2 = stops[i + 1]
            
            if temp1 <= temp <= temp2:
                # Calculate interpolation factor (0 to 1)
                factor = (temp - temp1) / (temp2 - temp1)
                
                # Convert hex to RGB
                r1, g1, b1 = int(color1[1:3], 16), int(color1[3:5], 16), int(color1[5:7], 16)
                r2, g2, b2 = int(color2[1:3], 16), int(color2[3:5], 16), int(color2[5:7], 16)
                
                # Interpolate in RGB space
                r = int(r1 + (r2 - r1) * factor)
                g = int(g1 + (g2 - g1) * factor)
                b = int(b1 + (b2 - b1) * factor)
                
                return WeatherColor(f"#{r:02x}{g:02x}{b:02x}")
        
        return WeatherColor("#808080" ) # Fallback gray
    
   
    def adjust_brightness(self, elevation: float, max_elev: float = 3000.0, range_pct: float = 0.10) -> 'WeatherColor':
        """Adjust color brightness based on elevation.
        
        Args:
            color: Hex color string
            elevation: Elevation in meters
            max_elev: Maximum elevation for normalization
            range_pct: Brightness adjustment range (0.10 = ±10%)
        """
        
        color = self.color
        
        # Normalize elevation to [-1, 1] range
        elev_norm = np.clip(elevation / max_elev, 0, 1) * 2 - 1
        
        # Adjust lightness: higher elevation = brighter (within range_pct)
        l = self.l + (elev_norm * range_pct)
        l = np.clip(l, 0, 1)
        
        # Convert back to RGB
        r, g, b = colorsys.hls_to_rgb(self.h, l, self.s)
        
        return  WeatherColor(f"#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}")
    
    
    def adjust_saturation(self, precip: float, climate_preset: ClimatePreset) -> 'WeatherColor':
        """Adjust color saturation based on precipitation.
        
        Args:
            color: Hex color string
            precip: Precipitation in mm/year
            climate_preset: ClimatePreset with precip_bins
        """
        color = self.color
        
        # Normalize precipitation using climate's bins
        bins = climate_preset.precip_bins
        if precip <= bins[0]:
            sat_factor = 0.5  # Very dry = 50% saturation
        elif precip >= bins[-1]:
            sat_factor = 1.0  # Very wet = 100% saturation
        else:
            # Linear interpolation between bins
            for i in range(len(bins) - 1):
                if bins[i] <= precip <= bins[i+1]:
                    sat_factor = 0.5 + 0.5 * ((precip - bins[i]) / (bins[i+1] - bins[i]))
                    break
        
       
        
        # Adjust saturation
        s = self.s * sat_factor
        s = np.clip(s, 0, 1)
        
        # Convert back to RGB
        r, g, b = colorsys.hls_to_rgb(self.h, self.l, s)
        
        return WeatherColor(f"#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}")

@patch
def render_icon_temperature(self: Terrain, ocean_color: str = "#2c5f8d") -> str:
    """Render hexes with inline fill based on temperature gradient.
    
    Args:
        ocean_color: Solid color for ocean hexes
    
    Returns:
        SVG string with solid-colored hexes
    """
    if 'temperature' not in self.fields:
        raise ValueError("Temperature field not computed")
    
    if 'precipitation' not in self.fields:
        raise ValueError("Precipitation field not computed")
    
    overlay = ""
    grid = self.hexGrid
    
    # Find max elevation for normalization
    land_elevs = self.elevations[self.elevations > 0]
    max_elev = land_elevs.max() if len(land_elevs) > 0 else 3000.0
    
    for i in range(len(self.elevations)):
        if self.elevations[i] <= 0:
            # Ocean
            color = ocean_color
        else:
            # Land: temperature gradient
            temp = self.fields['temperature'][i]
            color = WeatherColor.temp_to_color(temp)
            
            # Modulate by elevation (brightness)
            color = color.adjust_brightness( self.elevations[i], max_elev)
            
            # Modulate by precipitation (saturation)
            if self.climate:
                color = color.adjust_saturation( self.fields['precipitation'][i], self.climate)
        
        # Generate polygon with inline fill
        hex_obj = grid.hexes[i]
        points = " ".join([f"{p.x:.0f},{p.y:.0f}" for p in hex_obj.vertices()])
        overlay += f'\t<polygon points="{points}" fill="{color}"/>\n'
    
    return overlay

### Put it together

In [ ]:
@patch
def rainDemo(self:TerraDemo, scale=0.5, debug=True):
    # Load California with geographic bounds
    
    terrain = self.aussie_map()

    # Check that coordinates were computed
    if debug:
        print(f"Latitude range: {terrain.fields['latitude'].min():.2f}° to {terrain.fields['latitude'].max():.2f}°")
        print(f"Longitude range: {terrain.fields['longitude'].min():.2f}° to {terrain.fields['longitude'].max():.2f}°")

    terrain.compute_weather(debug = debug)
    smaller = terrain.shrinkWeather(scale)

    smaller.visualize_precipitation()
    return smaller.hexGrid.builder.show()


In [ ]:
??TerraDemo.bayArea_map


```python
@patch
def bayArea_map(self:TerraDemo):
    """Load Maui terrain with proper geographic bounds."""
    with resources.files('HexMagic').joinpath('data/templates/BayArea.txt').open() as f:
        terrain = Terrain.decode(f.read())

    # Set Maui's geographic bounds
    terrain.geo = GeoBounds(
        lat_min=37.57,   # 
        lat_max=38.03,   # 
        lon_min=-122.69, # 
        lon_max=-121.97  # 
    )

    # Compute hex coordinates
    terrain._compute_hex_coordinates()

    terrain.climate =  ClimatePreset(
            name='Mediterranean',
            lat_range=(30, 45),
            base_temp_range=(14, 18),
            wind_speed=12.0,
            wind_dir=270.0,  # Westerlies
            precip_base=0.08,
            nm=0.01,
            hw=2000.0,
            cw=0.0025,
            conv_time=1500.0,
            fall_time=1500.0,
            lapse_rate=6.5,
            description="Mild winters, dry summers. California-like."
        )

    return terrain
```

**File:** `/tmp/ipykernel_1211/4080233818.py`

In [ ]:
TerraDemo().rainDemo(.5)

Latitude range: -44.00° to -10.00°
Longitude range: 113.00° to 154.97°

Computing precipitation...
Grid: 70 x 70
Resolution: dx=59370m (~59.4km), dy=53979m (~54.0km)
Latitude: -44.0° to -10.0° (center: -27.0°)
Wind: 12.0 m/s from 270.0°
Background precip: 0.08 mm/h

Precipitation range: 1 - 4433 mm/year
Mean: 738 mm/year
Computing distance to coast...
Computing temperature...


In [ ]:
@patch
def rainbayAreaDemo(self:TerraDemo, scale=0.5, debug=True):
    # Load California with geographic bounds
    demo = TerraDemo()
    terrain = demo.bayArea_map()

    # Check that coordinates were computed
    if debug:
        print(f"Latitude range: {terrain.fields['latitude'].min():.2f}° to {terrain.fields['latitude'].max():.2f}°")
        print(f"Longitude range: {terrain.fields['longitude'].min():.2f}° to {terrain.fields['longitude'].max():.2f}°")

    terrain.compute_weather(debug = debug)
    smaller = terrain.shrinkWeather(scale)
    smaller.colorMap()
    smaller.hexGrid.update()

    smaller.add_rain_overlay()
    return smaller.hexGrid.builder.show()

In [ ]:
TerraDemo().rainbayAreaDemo(0.8)

In [ ]:
@patch
def mauiWeaterDemo(self:TerraDemo, scale=0.5, debug=True):
    # Load California with geographic bounds
    demo = TerraDemo()
    terrain = demo.maui_map()

    # Check that coordinates were computed
    if debug:
        print(f"Latitude range: {terrain.fields['latitude'].min():.2f}° to {terrain.fields['latitude'].max():.2f}°")
        print(f"Longitude range: {terrain.fields['longitude'].min():.2f}° to {terrain.fields['longitude'].max():.2f}°")

    terrain.compute_weather(debug = debug)
    smaller = terrain.shrinkWeather(scale)
    smaller.hexGrid.builder.adjust("icon",smaller.render_icon_temperature())
    #smaller.hexGrid.update()

    smaller.add_rain_overlay()
    return smaller.hexGrid.builder.show()

In [ ]:
TerraDemo().mauiWeaterDemo()

In [ ]:
!pwd